<a href="https://colab.research.google.com/github/ileunam/Prophet-calidad-de-aire-Curic-/blob/main/Prophet_calidad_de_aire_Curic%C3%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import IPython.core.display
import matplotlib

def apply_styles():
    matplotlib.rcParams['font.size'] = 12
    matplotlib.rcParams['figure.figsize'] = (18, 6)
    matplotlib.rcParams['lines.linewidth'] = 1

apply_styles()

In [50]:
%%html
<style type="text/css">
@import url('https://fonts.googleapis.com/css?family=Playfair+Display');


div.text_cell_render {font-family: 'Playfair Display', serif; color: #13213b; line-height: 145%; font-size:16px;}
</style>

In [51]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="ticks")

In [52]:
import warnings
warnings.filterwarnings('ignore')

In [101]:
import pandas as pd
url="https://sinca.mma.gob.cl/cgi-bin/APUB-MMA/apub.tsindico2.cgi?outtype=xcl&macro=./RVII/709/Cal/PM25//PM25.horario.horario.ic&from=120707&to=211130&path=/usr/airviro/data/CONAMA/&lang=esp&rsrc=&macropath="
df = pd.read_csv(url, sep=';', parse_dates = [1], header=None, skiprows=1, names=['Date', 'Time', 'Validated', 'Preliminaries', 'NotValidated', 'Unnamed'])
df.head(10)

,Date,Time,Validated,Preliminaries,NotValidated,Unnamed
0,120707,0100,NaN,NaN,NaN,NaN
1,120707,0200,NaN,NaN,NaN,NaN
2,120707,0300,NaN,NaN,NaN,NaN
3,120707,0400,NaN,NaN,NaN,NaN
4,120707,0500,NaN,NaN,NaN,NaN
5,120707,0600,NaN,NaN,NaN,NaN
6,120707,0700,NaN,NaN,NaN,NaN
7,120707,0800,NaN,NaN,NaN,NaN
8,120707,0900,NaN,NaN,NaN,NaN
9,120707,1000,NaN,NaN,NaN,NaN


Limpiar las fechas y horas

In [102]:
df['Date'] = pd.to_datetime(df['Date'], format='%y%m%d')
df['Time']  = pd.to_datetime(df['Time'], format='%H%M').apply(lambda x: str(x).split(' ')[1])
df.Date = df.Date.astype(str)
df['Datetime'] = pd.to_datetime(df['Date'] +' ' + df['Time'])
df.drop(['Date', 'Time', 'Unnamed'], axis='columns', inplace=True)
df.head(10)

,Validated,Preliminaries,NotValidated,Datetime
0,NaN,NaN,NaN,2012-07-07 01:00:00
1,NaN,NaN,NaN,2012-07-07 02:00:00
2,NaN,NaN,NaN,2012-07-07 03:00:00
3,NaN,NaN,NaN,2012-07-07 04:00:00
4,NaN,NaN,NaN,2012-07-07 05:00:00
5,NaN,NaN,NaN,2012-07-07 06:00:00
6,NaN,NaN,NaN,2012-07-07 07:00:00
7,NaN,NaN,NaN,2012-07-07 08:00:00
8,NaN,NaN,NaN,2012-07-07 09:00:00
9,NaN,NaN,NaN,2012-07-07 10:00:00


Limpiar los demás valores seleccionando el valor de entre los 3 que no esté indefinido. Primero prioriza los valores validados en caso de que hayan

In [107]:
import numpy as np
conditions = [
      ~pd.isnull(df['Validated']), 
      ~pd.isnull(df['Preliminaries']),
      ~pd.isnull(df['NotValidated'])
  ]

choices = [df['Validated'], df['Preliminaries'], df['NotValidated']]

df['Pollution'] = np.select(conditions, choices, default=np.nan)

df.sample(10)

,Validated,Preliminaries,NotValidated,Datetime,Pollution
26384,"19,1477",NaN,NaN,2015-07-11 09:00:00,"19,1477"
79386,NaN,75,NaN,2021-07-27 19:00:00,75
4826,NaN,NaN,NaN,2013-01-24 03:00:00,NaN
49920,3,NaN,NaN,2018-03-18 01:00:00,3
7868,109,NaN,NaN,2013-05-30 21:00:00,109
58849,17,NaN,NaN,2019-03-25 02:00:00,17
72033,NaN,26,NaN,2020-09-24 10:00:00,26
81615,NaN,5,NaN,2021-10-28 16:00:00,5
52422,27,NaN,NaN,2018-06-30 07:00:00,27
63960,10,NaN,NaN,2019-10-24 01:00:00,10


In [89]:
df['Pollution'] = df['Validated']
for index, row in df.iterrows():
    if pd.isnull(row['Validated']):
      pre    = row['Preliminaries']
      notval = row['Not-Validated']
      if not pd.isnull(pre):
        df.Pollution[str(index)] = pre
      else:
        df.Pollution[str(index)] = notval


KeyboardInterrupt: ignored